In [5]:
from utils import *
from sympy import symbols, trigsimp, lambdify, diff, init_printing
init_printing(use_unicode=True)

In [6]:
# Joint parameters declaration
q1, q2, a2, d1, d3 = symbols("q1 q2 a2, d1 d3", real=True)

In [7]:
# Forward Kinematics
H = Rz(q1)*Tz(d1)*Rx(q2)*Ty(a2)*Ty(d3)
H = trigsimp(H)
H

⎡cos(q₁)  -sin(q₁)⋅cos(q₂)  sin(q₁)⋅sin(q₂)    -(a₂ + d₃)⋅sin(q₁)⋅cos(q₂) ⎤
⎢                                                                         ⎥
⎢sin(q₁)  cos(q₁)⋅cos(q₂)   -sin(q₂)⋅cos(q₁)   (a₂ + d₃)⋅cos(q₁)⋅cos(q₂)  ⎥
⎢                                                                         ⎥
⎢   0         sin(q₂)           cos(q₂)       a₂⋅sin(q₂) + d₁ + d₃⋅sin(q₂)⎥
⎢                                                                         ⎥
⎣   0            0                 0                       1              ⎦

In [8]:
# General Rotation Matrix
R = H[0:3, :-1]
R

⎡cos(q₁)  -sin(q₁)⋅cos(q₂)  sin(q₁)⋅sin(q₂) ⎤
⎢                                           ⎥
⎢sin(q₁)  cos(q₁)⋅cos(q₂)   -sin(q₂)⋅cos(q₁)⎥
⎢                                           ⎥
⎣   0         sin(q₂)           cos(q₂)     ⎦

In [9]:
# General Translational Matrix
T = H[0:3, -1]
T

⎡ -(a₂ + d₃)⋅sin(q₁)⋅cos(q₂) ⎤
⎢                            ⎥
⎢ (a₂ + d₃)⋅cos(q₁)⋅cos(q₂)  ⎥
⎢                            ⎥
⎣a₂⋅sin(q₂) + d₁ + d₃⋅sin(q₂)⎦

In [10]:
# Calculating Jacobian Matrix
# Running this cell takes about 10 minutes
# Maybe Jump by 2 cells, to load the Jacobian Function from File
Jaco = Matrix([])

var = [q1, q2, d3]
for j in var:
    HD = trigsimp(diff(H, j))

    T1 = HD[0:3, -1]

    RD = HD[0:3, :-1]
    RW = trigsimp(RD*R.T)

    J = T1.row_insert(3, Matrix([RW[2,1], RW[0,2], RW[1,0]]))

    Jaco = Jaco.col_insert(len(Jaco), J)

Jaco

⎡(-a₂ - d₃)⋅cos(q₁)⋅cos(q₂)  (a₂ + d₃)⋅sin(q₁)⋅sin(q₂)   -sin(q₁)⋅cos(q₂)⎤
⎢                                                                        ⎥
⎢(-a₂ - d₃)⋅sin(q₁)⋅cos(q₂)  (-a₂ - d₃)⋅sin(q₂)⋅cos(q₁)  cos(q₁)⋅cos(q₂) ⎥
⎢                                                                        ⎥
⎢            0                   (a₂ + d₃)⋅cos(q₂)           sin(q₂)     ⎥
⎢                                                                        ⎥
⎢            0                        cos(q₁)                   0        ⎥
⎢                                                                        ⎥
⎢            0                        sin(q₁)                   0        ⎥
⎢                                                                        ⎥
⎣            1                           0                      0        ⎦

In [11]:
# Initialize time parameter
t = symbols("t")

# declare functions of joints
q1 = sin(t)
q2 = cos(2*t)
d3 = sin(3*t)

# differentiating the joint functions
dq1 = diff(q1, t)
dq2 = diff(q2, t)
dd3 = diff(d3, t)

# Matrix of Joint Velocities
dq = Matrix([dq1, dq2, dd3])

# Print Matrix
dq

⎡  cos(t)   ⎤
⎢           ⎥
⎢-2⋅sin(2⋅t)⎥
⎢           ⎥
⎣3⋅cos(3⋅t) ⎦

In [13]:
# Velocity of tool frame equal to product of Jacobian and derivative of joint parameters
v = trigsimp(Jaco * dq)

v

⎡(-2⋅a₂ - 2⋅d₃)⋅sin(q₁)⋅sin(q₂)⋅sin(2⋅t) + (-a₂ - d₃)⋅cos(q₁)⋅cos(q₂)⋅cos(t) -
⎢                                                                             
⎢(-a₂ - d₃)⋅sin(q₁)⋅cos(q₂)⋅cos(t) + (2⋅a₂ + 2⋅d₃)⋅sin(q₂)⋅sin(2⋅t)⋅cos(q₁) + 
⎢                                                                             
⎢                          (-2⋅a₂ - 2⋅d₃)⋅sin(2⋅t)⋅cos(q₂) + 3⋅sin(q₂)⋅cos(3⋅t
⎢                                                                             
⎢                                          -2⋅sin(2⋅t)⋅cos(q₁)                
⎢                                                                             
⎢                                          -2⋅sin(q₁)⋅sin(2⋅t)                
⎢                                                                             
⎣                                                 cos(t)                      

 3⋅sin(q₁)⋅cos(q₂)⋅cos(3⋅t)⎤
                           ⎥
3⋅cos(q₁)⋅cos(q₂)⋅cos(3⋅t) ⎥
                           ⎥
)             